In [ ]:
import tensorflow as tf
import numpy as np
import gym
import datetime
from tqdm import tqdm
import random
from go_ai import data, metrics, mcts, models, policies
import matplotlib.pyplot as plt
import os

# Hyperparameters

In [ ]:
BOARD_SIZE = 7

In [ ]:
ITERATIONS = 256
EPISODES_PER_ITERATION = 128
BATCH_SIZE = 32

In [ ]:
NUM_EVAL_GAMES = 32

In [ ]:
LEARNING_RATE = 2e-3

In [ ]:
WEIGHTS_DIR = 'model_weights/'
CHECKPOINT_PATH = WEIGHTS_DIR + 'checkpoint_{}x{}.h5'.format(BOARD_SIZE, BOARD_SIZE)
TMP_WEIGHTS_PATH = WEIGHTS_DIR + 'tmp.h5'
LOAD_SAVED_MODELS = False

In [ ]:
NUM_WORKERS = 4

In [ ]:
EPISODES_DIR = './data/'

# Go Environment
Train on a small board with heuristic reward for fast training and efficient debugging

In [ ]:
go_env = gym.make('gym_go:go-v0', size=BOARD_SIZE)

# Metrics and Tensorboard

In [ ]:
!rm -rf ./logs/

Metrics

In [ ]:
tb_metrics = {}
for metric_key in ['val_loss', 'overall_loss', 'move_loss']:
    tb_metrics[metric_key] = tf.keras.metrics.Mean('{}'.format(metric_key), 
                                                   dtype=tf.float32)
tb_metrics['pred_win_acc'] = tf.keras.metrics.Accuracy()

Tensorboard

In [ ]:
current_time = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
log_dir = 'logs/actor_critic/{}/main'.format(current_time)
summary_writer = tf.summary.create_file_writer(log_dir)    

# Machine Learning Models

In [ ]:
actor_critic = models.make_actor_critic(BOARD_SIZE, 'val_net', 'tanh')

In [ ]:
_ = tf.keras.utils.plot_model(actor_critic, to_file='logs/model.png')

In [ ]:
actor_critic.summary()

In [ ]:
opponent = tf.keras.models.clone_model(actor_critic)

In [ ]:
if LOAD_SAVED_MODELS:
    actor_critic.load_weights(CHECKPOINT_PATH)
    opponent.load_weights(CHECKPOINT_PATH)
    print("Loaded saved models")

# Policies

In [ ]:
temp_policy_args = {
    'mode': 'actor_critic',
    'model_path': TMP_WEIGHTS_PATH,
}

In [ ]:
checkpoint_policy_args = {
    'mode': 'actor_critic',
    'model_path': CHECKPOINT_PATH,
}

In [ ]:
random_policy_args = {
    'mode': 'random'
}

In [ ]:
greedy_policy_args = {
    'mode': 'greedy'
}

# Value Function

In [ ]:
checkpoint_val_func_args = {
    'weights_path': CHECKPOINT_PATH
}

# Demo Trajectories

Symmetries

In [ ]:
%%time
metrics.plot_symmetries(go_env, actor_critic, 'logs/symmetries.jpg')

Plot a whole game trajectory

In [ ]:
%%time
my_policy = policies.ActorCriticPolicy(actor_critic)
black_won, traj = data.self_play(go_env, policy=my_policy, get_memory=True)
print(f"{len(traj)} steps")

In [ ]:
%%time
replay_mem = []
actor_critic = models.make_actor_critic(go_env.size)
val_func = models.make_val_func(actor_critic)
data.add_traj_to_replay_mem(replay_mem, black_won, traj, val_func, add_symmetries=False)
fig = metrics.state_responses(actor_critic, replay_mem)
fig.savefig('logs/a_trajectory.jpg')
plt.close()

# Train

In [ ]:
actor_critic_opt = tf.keras.optimizers.Adam(LEARNING_RATE)

In [ ]:
for iteration in range(ITERATIONS):
    # Make and write out the episode data
    data.make_episodes(BOARD_SIZE, checkpoint_policy_args, checkpoint_policy_args, checkpoint_val_func_args,
                       EPISODES_PER_ITERATION, num_workers=NUM_WORKERS, outdir=EPISODES_DIR)
    # Read in the episode data
    np_data = data.episodes_from_dir(EPISODES_DIR)
    batched_np_data = [np.array_split(datum, len(np_data[0]) // BATCH_SIZE) for datum in np_data]
    batched_mem = list(zip(*batched_np_data))
    
    # Optimize
    models.update_win_prediction(actor_critic, batched_mem, actor_critic_opt, iteration, tb_metrics)
    
    # Save weights for evaluation
    actor_critic.save_weights(TMP_WEIGHTS_PATH)
    
    # Evaluate against checkpoint model and other baselines
    rand_win_rate = data.make_episodes(BOARD_SIZE, temp_policy_args, random_policy_args, 
                                       checkpoint_val_func_args, NUM_EVAL_GAMES, num_workers=NUM_WORKERS)
    greed_win_rate = data.make_episodes(BOARD_SIZE, temp_policy_args, greedy_policy_args, 
                                        checkpoint_val_func_args, NUM_EVAL_GAMES, num_workers=NUM_WORKERS)
    opp_win_rate = data.make_episodes(BOARD_SIZE, temp_policy_args, checkpoint_policy_args, 
                                      checkpoint_val_func_args, NUM_EVAL_GAMES, num_workers=NUM_WORKERS)

    stats = "{:.1f}%O, {:.1f}%R, {:.1f}%G".format(100*opp_win_rate, 
                                                 100*rand_win_rate,
                                                 100*greed_win_rate)
    
    # If it's better than the checkpoint, update
    if opp_win_rate > 0.6:
        actor_critic.save_weights(CHECKPOINT_PATH)        
        print("{} Accepted new model".format(stats))
    else:
        print("{} Rejected new model".format(stats))
    if os.path.exists(CHECKPOINT_PATH):
        opponent.load_weights(CHECKPOINT_PATH)
        actor_critic.load_weights(CHECKPOINT_PATH)
    
    # Log results and resets the metrics
    metrics.log_to_tensorboard(summary_writer, tb_metrics, iteration, go_env, 
                               actor_critic, 'logs/a_trajectory.jpg')

# Evaluate

Play against our AI

In [ ]:
go_env = gym.make('gym_go:go-v0', size=BOARD_SIZE)
data.pit(go_env, black_policy=opponent_policy, white_policy=human_policy)